In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

In [4]:
counties_df = pd.read_csv('results/counties.csv')
counties_df

,CO_FIPS,CO_NAME
0,1,Beaver
1,3,Box Elder
2,5,Cache
3,7,Carbon
4,9,Daggett
5,11,Davis
6,13,Duchesne
7,15,Emery
8,17,Garfield
9,19,Grand


In [5]:
prv_taz_percent_df = pd.read_csv("data/prev-v911/Lookup - BYTAZAgePct - AllCo.csv")
prv_taz_percent_df.rename(columns={';CO_TAZID':'CO_TAZID'}, inplace=True)
prv_taz_percent_df

,CO_TAZID,PCT_SUM,PCT_0TO17,PCT_18TO64,PCT_65P,SUBAREAID,CO_FIPS,CO_NAME
0,1001,1,0.30,0.53,0.17,0,1,BEAVER
1,1002,1,0.35,0.44,0.21,0,1,BEAVER
2,1003,1,0.41,0.40,0.19,0,1,BEAVER
3,1004,1,0.30,0.53,0.17,0,1,BEAVER
4,1005,1,0.27,0.52,0.21,0,1,BEAVER
...,...,...,...,...,...,...,...,...
9810,570424,1,0.13,0.55,0.32,1,57,WEBER
9811,570425,1,0.13,0.55,0.32,1,57,WEBER
9812,570426,1,0.13,0.55,0.32,1,57,WEBER
9813,570427,1,0.28,0.60,0.12,1,57,WEBER


In [6]:
new_taz_percent_df = pd.read_csv("results/Lookup - BYTAZAgePct - AllCo.csv")
new_taz_percent_df

,CO_TAZID,PCT_SUM,PCT_0TO17,PCT_18TO64,PCT_65P,SOURCE,CO_FIPS
0,1001.0,1.0,0.30,0.51,0.19,tract,1
1,1002.0,1.0,0.30,0.51,0.19,tract,1
2,1003.0,1.0,0.41,0.40,0.19,block,1
3,1004.0,1.0,0.30,0.51,0.19,tract,1
4,1005.0,1.0,0.27,0.52,0.21,block,1
...,...,...,...,...,...,...,...
9951,570424.0,1.0,0.28,0.56,0.16,tract,57
9952,570425.0,1.0,0.14,0.67,0.19,block,57
9953,570426.0,1.0,0.23,0.58,0.19,tract,57
9954,570427.0,1.0,0.18,0.53,0.29,block,57


# Charts

In [7]:


# Dropdown for selecting county
county_dropdown = widgets.Dropdown(
    options=[(row['CO_NAME'], row['CO_FIPS']) for _, row in counties_df.iterrows()],
    description='Select County:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Fields to plot
fields = [
    ('% Age 0-17', 'PCT_0TO17'),
    ('% Age 18-64', 'PCT_18TO64'),
    ('% Age 65+', 'PCT_65P')
]

# Plot update function
def update_charts(co_fips):

    county_name = counties_df.loc[counties_df['CO_FIPS'] == co_fips, 'CO_NAME'].values[0]

    prv_filtered = prv_taz_percent_df[prv_taz_percent_df['CO_FIPS'] == co_fips]
    new_filtered = new_taz_percent_df[new_taz_percent_df['CO_FIPS'] == co_fips]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=[label for label, _ in fields],
        horizontal_spacing=0.05
    )

    for i, (label, field_name) in enumerate(fields, start=1):
        merged = pd.merge(
            prv_filtered[['CO_TAZID', field_name]],
            new_filtered[['CO_TAZID', field_name]],
            on='CO_TAZID',
            suffixes=('_prev', '_new')
        )

        fig.add_trace(
            go.Scatter(
                x=merged[f'{field_name}_prev'],
                y=merged[f'{field_name}_new'],
                mode='markers',
                name=label,
                text=merged['CO_TAZID'],
                marker=dict(size=8, opacity=0.7),
                showlegend=False
            ),
            row=1, col=i
        )

        # Reference line: y = x
        fig.add_trace(
            go.Scatter(
                x=[0, 1], y=[0, 1],
                mode='lines',
                line=dict(dash='dash', color='gray'),
                showlegend=False
            ),
            row=1, col=i
        )

        fig.update_xaxes(
            range=[0, 1.0], title_text='Previous',
            scaleanchor=f'y{i}', row=1, col=i
        )
        fig.update_yaxes(
            range=[0, 1.0], title_text='New',
            scaleratio=1, row=1, col=i
        )

    fig.update_layout(
        title_text=f"Change in Age Group % for {county_name} County",
        height=600,
        width=1800,
        margin=dict(t=60, l=40, r=40, b=40)
    )

    fig.show()

# Display dropdown + interactive plot
ui = widgets.VBox([county_dropdown])
out = widgets.interactive_output(update_charts, {
    'co_fips': county_dropdown
})

display(ui, out)


Output()